In [ ]:
# installing required packages
# On local machine, please install all of the requirements
# with `python>=3.10`

%pip install transformers

In [ ]:
# loading the model and tokenizer

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM as AutoModel

CKPT = "gyulukeyi/nallm-bart"
model = AutoModel.from_pretrained(CKPT)
tokenizer = AutoTokenizer.from_pretrained(CKPT)

In [ ]:
# setting up the device
# BART inference works well with 'cpu'
# but for inference on batched texts, 'cuda' acceleartion is better
# Optionally use 'mps' on MacOS with Apple Silicon

import torch

device = 'cuda' if torch.cuda.is_available else \
            'mps' if torch.backends.mps.is_available else \
            'cpu'

print("Will use: " + device)

model = model.to(device)

In [ ]:
# set up text
# ensure to format the input text for optimal performance
# (the format made is the format given to the model while fine-tuning)

input = {
    "organization": "경찰청",
    "title": "행정심판의 대상",
    "question": "행정심판의 대상은 무엇인가요?"
}

input = f"<{input['organization']}> {input['title']}\n{input['question']}"
print(input)

In [ ]:
# generate

out = model.generate(
    input_ids = torch.tensor([tokenizer(input).input_ids]).to(device),
    attention_mask = torch.tensor([tokenizer(input).attention_mask]).to(device),
    max_new_tokens=120,
    top_k=10,
    repetition_penalty=1.2,
)

out = tokenizer.decode(out.squeeze(0),
             skip_special_tokens=True, clean_up_tokenization_spaces=True)
print(out)